In [ ]:
pip install streamlit


In [ ]:
!pip install matplotlib


In [ ]:
!pip install folium

In [ ]:
!pip install streamlit_folium


In [ ]:
streamlit run app.py

In [ ]:
pip install streamlit streamlit-folium geopy


In [25]:
!streamlit --version


Streamlit, version 1.42.1


In [8]:
%%writefile app.py
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
import folium
from datetime import datetime, timedelta
from streamlit_folium import st_folium
from sklearn.linear_model import LinearRegression
from geopy.geocoders import Nominatim
from IPython.display import FileLink

# -------------------------------
# Page Config and Custom CSS
# -------------------------------
st.set_page_config(page_title="Air Quality Monitor & Predictor", layout="wide", initial_sidebar_state="expanded")
st.markdown("""
    <style>
    .main { 
        background-color: #f4f4f9;
        padding: 1rem;
    }
    .sidebar .sidebar-content {
        background-color: #e8eaf6;
        padding: 1rem;
    }
    .stButton>button {
        background-color: #1976d2;
        color: white;
        border-radius: 5px;
        border: none;
        padding: 0.5em 1em;
    }
    </style>
    """, unsafe_allow_html=True)

# -------------------------------
# API KEYS
# -------------------------------
# Removed hardcoded API keys to protect privacy.
# The code now expects the keys to be available as environment variables.
WAQI_API_KEY = os.getenv("WAQI_API_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

# If you prefer to load API keys from a file, uncomment the following lines and
# create an "api_keys.txt" file with the format: KEY_NAME=actual_key
# def load_api_keys():
#     with open("api_keys.txt", "r") as file:
#         keys = {}
#         for line in file:
#             name, key = line.strip().split("=")
#             keys[name] = key
#         return keys
#
# api_keys = load_api_keys()
# WAQI_API_KEY = api_keys["WAQI_API_KEY"]
# OPENWEATHER_API_KEY = api_keys["OPENWEATHER_API_KEY"]

# -------------------------------
# Indian States & UTs with default center coordinates
# -------------------------------
states_info = {
    "Andhra Pradesh": (15.9129, 79.7400),
    "Arunachal Pradesh": (28.2180, 94.7278),
    "Assam": (26.2006, 92.9376),
    "Bihar": (25.0961, 85.3131),
    "Chhattisgarh": (21.2787, 81.8661),
    "Goa": (15.2993, 74.1240),
    "Gujarat": (22.2587, 71.1924),
    "Haryana": (29.0588, 76.0856),
    "Himachal Pradesh": (31.1048, 77.1734),
    "Jharkhand": (23.6102, 85.2799),
    "Karnataka": (15.3173, 75.7139),
    "Kerala": (8.5241, 76.9366),  # Thiruvananthapuram region
    "Madhya Pradesh": (23.2599, 77.4126),
    "Maharashtra": (19.7515, 75.7139),
    "Manipur": (24.6637, 93.9063),
    "Meghalaya": (25.4670, 91.3662),
    "Mizoram": (23.1645, 92.9376),
    "Nagaland": (26.1584, 94.5624),
    "Odisha": (20.9517, 85.0985),
    "Punjab": (31.1471, 75.3412),
    "Rajasthan": (27.0238, 74.2179),
    "Sikkim": (27.5330, 88.5122),
    "Tamil Nadu": (11.1271, 78.6569),
    "Telangana": (18.1124, 79.0193),
    "Tripura": (23.9408, 91.9882),
    "Uttar Pradesh": (26.8467, 80.9462),
    "Uttarakhand": (30.0668, 79.0193),
    "West Bengal": (22.9868, 87.8550),
    "Andaman & Nicobar Islands": (11.7401, 92.6586),
    "Chandigarh": (30.7333, 76.7794),
    "Dadra & Nagar Haveli": (20.1809, 73.0169),
    "Daman & Diu": (20.3974, 72.8328),
    "Lakshadweep": (10.3280, 72.7846),
    "Delhi": (28.7041, 77.1025),
    "Puducherry": (11.9416, 79.8083),
    "Jammu & Kashmir": (33.7782, 76.5762),
    "Ladakh": (34.1526, 77.5770)
}

# -------------------------------
# Reverse Geocoding Setup
# -------------------------------
geolocator = Nominatim(user_agent="streamlit_app")

def get_place_name(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en', timeout=10)
        if location:
            address = location.raw.get('address', {})
            place = address.get('city', None) or address.get('town', None) or address.get('village', None) or address.get('state', None)
            return place
    except Exception:
        return None

def get_state_from_coordinates(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en', timeout=10)
        if location:
            return location.raw.get('address', {}).get('state', None)
    except Exception:
        return None

# -------------------------------
# Function: Fetch Live Data from APIs
# -------------------------------
def get_air_quality_data(lat, lon):
    waqi_url = f"https://api.waqi.info/feed/geo:{lat};{lon}/?token={WAQI_API_KEY}"
    openweather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&units=metric"
    
    try:
        waqi_response = requests.get(waqi_url).json()
        aqi = waqi_response.get("data", {}).get("aqi", "N/A")
        pm25 = waqi_response.get("data", {}).get("iaqi", {}).get("pm25", {}).get("v", "N/A")
    except Exception:
        aqi, pm25 = "N/A", "N/A"
    
    try:
        weather_response = requests.get(openweather_url).json()
        temperature = weather_response.get("main", {}).get("temp", "N/A")
        humidity = weather_response.get("main", {}).get("humidity", "N/A")
        wind_speed = weather_response.get("wind", {}).get("speed", "N/A")
        if temperature != "N/A":
            temperature = float(temperature)
        if humidity != "N/A":
            humidity = float(humidity)
        if wind_speed != "N/A":
            wind_speed = float(wind_speed)
    except Exception:
        temperature, humidity, wind_speed = "N/A", "N/A", "N/A"
    
    return {"AQI": aqi, "PM2.5": pm25, "Temperature": temperature, "Humidity": humidity, "Wind Speed": wind_speed}

# -------------------------------
# Callback: Update Coordinates on State Change
# -------------------------------
def update_coordinates():
    state = st.session_state['selected_state']
    st.session_state['lat'] = states_info[state][0]
    st.session_state['lon'] = states_info[state][1]

# -------------------------------
# Main App Layout
# -------------------------------
st.title("Live Air Quality Monitoring & Future Prediction")
st.markdown("### Monitor air quality in India with live data and future predictions")
st.markdown("---")

# Sidebar for mode selection and settings
mode = st.sidebar.radio("Select Mode", ["Live Monitoring", "Future Prediction"])
st.sidebar.markdown("### Settings")

# Auto-update state based on coordinates
if "auto_state" not in st.session_state:
    st.session_state["auto_state"] = sorted(list(states_info.keys()))[0]

state_list = sorted(list(states_info.keys()))
default_index = state_list.index(st.session_state["auto_state"]) if st.session_state["auto_state"] in state_list else 0
selected_state = st.sidebar.selectbox("Select a State/UT", state_list, index=default_index, key='selected_state', on_change=update_coordinates)
lat = st.sidebar.number_input("Latitude", value=states_info[selected_state][0], format="%.4f")
lon = st.sidebar.number_input("Longitude", value=states_info[selected_state][1], format="%.4f")

# Automatically update auto_state if reverse geocoding finds a different state
new_state = get_state_from_coordinates(lat, lon)
if new_state:
    st.session_state["auto_state"] = new_state

# -------------------------------
# Content: Live Monitoring and Future Prediction
# -------------------------------
if mode == "Live Monitoring":
    st.subheader("Live Air Quality Monitoring")
    
    # Use clicked coordinates if available; otherwise default to sidebar coordinates.
    if "clicked_coords" in st.session_state:
        current_lat, current_lon = st.session_state["clicked_coords"]
    else:
        current_lat, current_lon = lat, lon

    # Create the interactive map centered at current coordinates.
    m = folium.Map(location=[current_lat, current_lon], zoom_start=7)
    
    # Render the interactive map and capture click events.
    map_data = st_folium(m, width=700, height=500)
    
    # If a left-click is detected, update session state with new coordinates.
    if map_data and map_data.get("last_clicked") is not None:
        new_clicked_coords = (map_data["last_clicked"]["lat"], map_data["last_clicked"]["lng"])
        st.session_state["clicked_coords"] = new_clicked_coords

    # Display subheading and metrics based on the current coordinates.
    if "clicked_coords" in st.session_state:
        clicked_lat, clicked_lon = st.session_state["clicked_coords"]
        place_name = get_place_name(clicked_lat, clicked_lon)
        st.subheader(f"Live Data for {place_name} at ({clicked_lat:.4f}, {clicked_lon:.4f})")
        live_data = get_air_quality_data(clicked_lat, clicked_lon)
        folium.Marker(location=[clicked_lat, clicked_lon], popup=place_name, icon=folium.Icon(color='red')).add_to(m)
    else:
        current_place = get_place_name(lat, lon)
        st.subheader(f"Live Data for {current_place} at ({lat:.4f}, {lon:.4f})")
        live_data = get_air_quality_data(lat, lon)
        folium.Marker(location=[lat, lon], popup=current_place, icon=folium.Icon(color='blue')).add_to(m)
    
    # Display live data metrics.
    col1, col2, col3, col4, col5 = st.columns(5)
    col1.metric("AQI", live_data["AQI"])
    col2.metric("PM2.5", live_data["PM2.5"])
    col3.metric("Temp (°C)", live_data["Temperature"])
    col4.metric("Humidity (%)", live_data["Humidity"])
    col5.metric("Wind (m/s)", live_data["Wind Speed"])
    
    # Render the same interactive map with updated marker.
    st_folium(m, width=700, height=500)

elif mode == "Future Prediction":
    # Use clicked coordinates if available; otherwise default to sidebar coordinates.
    if "clicked_coords" in st.session_state:
        pred_lat, pred_lon = st.session_state["clicked_coords"]
    else:
        pred_lat, pred_lon = lat, lon

    place_name_future = get_place_name(pred_lat, pred_lon)
    st.header(f"Future AQI Prediction for {place_name_future} at ({pred_lat:.4f}, {pred_lon:.4f})")
    st.info("Simulating 180 days of historical AQI data to train a prediction model.")
    n_days = 180
    base_date = datetime.today() - timedelta(days=n_days)
    dates = [base_date + timedelta(days=i) for i in range(n_days)]
    simulated_aqi = 100 + 50 * np.sin(np.linspace(0, np.pi, n_days)) + np.random.normal(0, 10, n_days)
    sim_df = pd.DataFrame({"Date": dates, "AQI": simulated_aqi})
    sim_df['DayIndex'] = np.arange(n_days)
    
    X_sim = sim_df[['DayIndex']]
    y_sim = sim_df['AQI']
    ml_model = LinearRegression()
    ml_model.fit(X_sim, y_sim)
    
    st.line_chart(sim_df.set_index("Date"))
    st.write("Trained a simple Linear Regression model on simulated AQI data.")
    
    future_date = st.date_input("Select a Future Date", datetime.today() + timedelta(days=1))
    day_index_future = (datetime.combine(future_date, datetime.min.time()) - base_date).days
    predicted_aqi = ml_model.predict([[day_index_future]])[0]
    st.write(f"**Predicted AQI for {future_date.strftime('%Y-%m-%d')}:** {predicted_aqi:.2f}")
    
    plt.figure(figsize=(10, 5))
    plt.plot(sim_df["Date"], sim_df["AQI"], label="Historical AQI")
    plt.scatter([future_date], [predicted_aqi], color="red", label="Predicted AQI", zorder=5)
    plt.xlabel("Date")
    plt.ylabel("AQI")
    plt.title("Historical & Predicted AQI")
    plt.legend()
    st.pyplot(plt)

# -------------------------------
# CSV Export Option for Live Data
# -------------------------------
if st.sidebar.checkbox("Export Live Data for All Regions"):
    all_data = []
    for state, (lat, lon) in states_info.items():
        row = [state, lat, lon]
        d = get_air_quality_data(lat, lon)
        row.extend([d["AQI"], d["PM2.5"], d["Temperature"], d["Humidity"], d["Wind Speed"]])
        all_data.append(row)
    live_df = pd.DataFrame(all_data, columns=["State", "Latitude", "Longitude", "AQI", "PM2.5", "Temperature", "Humidity", "Wind Speed"])
    documents_folder = r"C:\Users\user\Documents"
    csv_filename = os.path.join(documents_folder, "india_air_quality.csv")
    live_df.to_csv(csv_filename, index=False)
    st.write(f"✅ Live data saved at: {csv_filename}")
    try:
        st.markdown(FileLink(csv_filename, result_html_prefix="Download CSV: ")._repr_html_(), unsafe_allow_html=True)
    except Exception:
        st.write("Please open the file manually from your Documents folder.")


Overwriting app.py
